# Filters
They are a single application applied to the entire image, and they serve various purposes.
They are usually needed in order to highlight a feature in the image.

Essentially, a filter is a matrix, called **kernel** that is applied to every single pixel in the image.  
A different value of the **stride** can make it apply on a fraction of the pixels.

In [5]:
import cv2
import numpy as np

img = cv2.imread("samples/tree.png")
#cv2.imshow("Original", img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

It is strongly suggested to use *odd-sized* (square) kernels, because they have a well - defined "center".

In [14]:
kernel = np.array([
    [1, 0, 1],
    [1, 0, 1],
    [1, 0, 1]
], np.float32)
#Applying kernel
filtered = cv2.filter2D(img, -1, kernel)
#filter2D is the method for applying this kind of transformations.
#the second parameter is the number of channels we want in the output image:
#-1 sets it to the number of channels of the original image
#This is because if we want to apply the kernel to be applied to every channel,
#It must be specified.

cv2.imshow("Filtered", filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Blur

It is useful to *denoise* an image, since it averages the values accross pixels.

In [16]:
blurred = cv2.blur(img, (10, 10), None)
cv2.imshow("Blurred", blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

The second parameter of `blur` is the dimension of the kernel. The bigger the kernel, the more blurred the image will be.

## Gaussian blur

It assumes a gaussian distribution of the noise, and the kernel is arranged accordingly.

In [18]:
gaussian_blurred = cv2.GaussianBlur(img, (3, 3), 1)
cv2.imshow("Blurred", gaussian_blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

`sigmaX` and `sigmaY` set the standard deviation of the noise (kernel).  
Setting a single value is giving the same value to both parameters.

## Median blur

The so-called *salt and pepper noise* is a kind of noise that can be removed through this filter.  
It is made up of completely white and black pixels.

In [20]:
saltpepper = cv2.imread("samples/saltpepper.jpeg")
median_blur = cv2.medianBlur(saltpepper, 5, None)
cv2.imshow("Blurred", median_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In this case, there is a single value in the `medianBlur` method since the kernel **must** be square.

## Bilateral filter

This keeps the edges sharp while still applying some noise reduction inside shapes.

In [23]:
bilateraled = cv2.bilateralFilter(img, 9, 75, 75)
cv2.imshow("Blurred", bilateraled)
cv2.waitKey(0)
cv2.destroyAllWindows()

Parameters???

# Sharpening

## Sharpen mask

It sums the original with a blurred version in order to sharpen the edges.

In [33]:
img = cv2.imread("samples/tiger.jpg")
blurred = cv2.GaussianBlur(img, (7, 7), 10)
sharped_img = cv2.addWeighted(img, 0.7, blurred, 0.3, 0)
cv2.imshow("Sharpen mask", sharped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

`addWeighted` performs a weighted mean of the two images. The last parameter performs some color correction.

## Sharpen kernel

It just applies a well-known kernel to sharpen it.

In [31]:
kernel = np.array([
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]
], np.float32)

sharpened = cv2.filter2D(img, -1, kernel)
cv2.imshow("Kernel sharpening", sharpened)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Derivative - based filters

## Sobel operator

Contours can be interpreted as sharp changes, and if we treat pixel as functions, they can be detected via derivatives. There are 2 derivatives (one for the x - axis, and one for the y - axis). To do it, it is sufficient to transpose the kernel.  
Once the partial derivatives are calculated, they can be combined into a single gradient.  
Since every pixel ranges in $[0, 255]$, they must also be normalized.

In [37]:
#First of all, let's convert the image to greyscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

der_x = cv2.Sobel(img_gray, -1, 1, 0)
der_y = cv2.Sobel(img_gray, -1, 0, 1)

scaled_x = cv2.convertScaleAbs(der_x)
scaled_y = cv2.convertScaleAbs(der_y)
'''
cv2.imshow("Scaled-x", scaled_x)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("Scaled-y", scaled_y)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

out = cv2.addWeighted(scaled_x, 0.5, scaled_y, 0.5, 0)
cv2.imshow("Total", out)
cv2.waitKey(0)
cv2.destroyAllWindows()


The last two arguments are booleans expressing which derivative we are taking. They can be easily combined with a weighted average.

## Laplacian filter

It applies twice the Sobel operator, thus providing a second-derivative overview of the "zero" points.  
Consequently, it also contains all the borders extracted by the Sobel operator.

In [39]:
der = cv2.Laplacian(img_gray, -1, (3, 3))

cv2.imshow("Total", der)
cv2.waitKey(0)
cv2.destroyAllWindows()


# A cartoonized filter

In [46]:
import cv2
import numpy as np
img = cv2.imread("samples/tiger.jpg")

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#apply a light blur for cleaning the image
img_gray = cv2.medianBlur(img, 5)
#extract the contours
contours = cv2.Laplacian(img_gray, cv2.CV_8U, ksize=5) #this parameter creates an 8-bit unsigned image
# so it takes a value of grey between 0 and 255
#applying a threshold so we can take only strong (true) edges
ret, thresholded = cv2.threshold(contours, 70, 255, cv2.THRESH_BINARY_INV)
#the last parameter specifies the type of colorscheme used in this image
#THRESH_BINARY_INV has black background and white contours

#using the bilateral filter with high values to get the colours:
colours = cv2.bilateralFilter(img, 10, 250, 250)
#converting the gray edges image into a 3-channel image
edges = cv2.cvtColor(thresholded, code = cv2.COLOR_GRAY2BGR)

#bitwise AND for merging edges and colours
out = cv2.bitwise_and(colours, img)
cv2.imshow("Cartoonized img", out)
cv2.waitKey(0)
cv2.destroyAllWindows()



error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<1>; VDcn = cv::impl::{anonymous}::Set<3, 4>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 3
